# ChatGPT API

Adapted from https://github.com/simecek/mlprague2024/blob/main/04_ChatGPT_API.ipynb


In [ ]:
api_key = "sk-********" ## You must change this to actual API key distributed during

In [ ]:
from datasets import load_dataset

dataset = load_dataset('simecek/synczech50', split='train')

dataset

In [ ]:
dataset[0]

In [ ]:
def get_prompt(x):
    question, options = x['question'], (x['optionA'], x['optionB'], x['optionC'], x['optionD'])
    text = f"""Which option A/B/C/D is the best answer for the question. Answer with one letter, no explanation.

Question (in Czech): {question}

Options:
A) {options[0]}
B) {options[1]}
C) {options[2]}
D) {options[3]}

Answer (just 1 letter, A/B/C/D):"""
    return text

prompts = [get_prompt(x) for x in dataset]

print(prompts[0])

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
)

In [ ]:
def get_answer(prompt):

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ],
      temperature=0,
      max_tokens=3)

    return response.choices[0].message.content.strip().upper()[:1]


get_answer(prompts[40])


In [ ]:
from tqdm.notebook import tqdm

answers = []

for prompt in tqdm(prompts):
  a = get_answer(prompt)
  answers.append(a)


In [ ]:
correct = 0

for answer, solution in zip(answers, dataset['solution']):
  if answer == solution:
    correct += 1

# 80% for GPT3.5 (11 mistakes)
# 94% for GPT4 (3 mistakes)
correct / len(answers)

In [ ]:
# List the mistakes

for answer, x in zip(answers, dataset):
  if answer != x['solution']:
    print(answer, str(x))
